# Frank-Wolfe Algorithm

*Mar, 2017 by K.Wu*

In this report, network are presented as a CSV file:

- [**/data/Frank-Wolfe-algo-data.csv**](https://github.com/wklchris/Reports/blob/master/data/Frank-Wolfe-algo-data.csv): All links of the network.

First, here are the packages needed in this report. Install: 

- Python 3
- numpy package
- pandas package

to continue.

In [1]:
import os
import numpy as np
import pandas as pd

from matplotlib import pyplot as plt
from matplotlib.animation import FuncAnimation

## Introduction

In this report, we have a network like below:

<img src="https://raw.githubusercontent.com/wklchris/Reports/master/pics/FrankWolfe-network.png" alt="Network Picture" style="width: 500px;"/>

The free flow travel time $t_a$ and capacities $c_a$ for each type of link are:

- Black links: $t_a = 10, c_a = 50$
- Red links: $t_a = 30, c_a = 200$
- Yellow links: $t_a = 15, c_a = 200$
- Cyan links: $t_a = 0, c_a = 2000$

We also have the O-D table of this network, shown as below:

In [2]:
ODarr = pd.DataFrame([[100, 200, 300, 50],
                      [100, 200, 200, 100],
                      [300, 50, 100, 100],
                      [200, 500, 50, 200]])

# Rename columns & rows
ODarr.columns = list(range(201, 205))
ODarr.index = list(range(101, 105))

ODarr

,201,202,203,204
101,100,200,300,50
102,100,200,200,100
103,300,50,100,100
104,200,500,50,200


## Network Representation



In [3]:
linkarr = pd.read_csv(r"{}/data/Frank-Wolfe-algo-data.csv".format(os.getcwd()))
linkarr.head()

,start,end,type
0,1,2,black
1,2,3,black
2,3,4,black
3,14,15,black
4,15,16,black


We can modify the link array according to the type:

In [4]:
linkarr["fftime"] = 0
linkarr["cap"] = 0

# Set the FreeFlowTraveltime and Capacity of each link
linkarr.loc[linkarr.type == "black", "fftime"] = 10
linkarr.loc[linkarr.type == "black", "cap"] = 50
linkarr.loc[linkarr.type == "yellow", "fftime"] = 15
linkarr.loc[linkarr.type == "yellow", "cap"] = 200
linkarr.loc[linkarr.type == "red", "fftime"] = 30
linkarr.loc[linkarr.type == "red", "cap"] = 200
linkarr.loc[linkarr.type == "cyan", "fftime"] = 0
linkarr.loc[linkarr.type == "cyan", "cap"] = 2000

linkarr.head(3)

,start,end,type,fftime,cap
0,1,2,black,10,50
1,2,3,black,10,50
2,3,4,black,10,50


## Application Scenarios

As for the costs, we use BPR function to compute the cost on each link. Cost on link $a$, called $t_a$, is a function of $x_a$:

$$ t_a(x_a) = t_a^0 \left[1 + \alpha\left(\frac{x_a}{c_a'}\right)^\beta\right] $$

where $\alpha = 0.15, \beta = 4, c_a' = 0.9c_a$. And we can define it as a function:

In [5]:
def bpr(xa, fft, ca, alpha=0.15, beta=4, ca_factor=0.9):
    """
    Compute the cost when the flow is 'xa' with BPR function.
    """
    return np.multiply(fft, 1 + alpha * (np.divide(xa, ca_factor * ca) ** beta))

As aforementioned, $\beta = 4$. So we need a function to search the numerical solution for quadratic equation:

In [6]:
def solvex4eq(q, theta=0.001, a=0, b=1):
    """
    Bi-section method to solve quartic equation.
    """
    y = lambda x: q[0] + q[1] * x + q[2] * x ** 2 + q[3] * x ** 3 + q[4] * x ** 4
    fa = y(a)
    fb = y(b)
    
    if fa * fb > 0:
        return list(map(y, np.linspace(a, b, 20)))
    elif (fa == 0):
        return a
    elif (fb == 0):
        return b
    
    while (b - a > theta):
        c = (b + a) / 2
        fc = y(c)
        if (fc == 0):
            return c
        elif (fc * fa < 0):
            b = c
        elif (fc * fb < 0):
            a = c
    
    return c

# Example: (x + 1)^3 * (x - 0.2) = 0
solvex4eq([-0.2, 0.4, 2.4, 2.8, 1])

0.2001953125

As for the shortest path finding, we use the function defined in the [Label-correcting-algo.ipynb](https://github.com/wklchris/Reports/blob/master/Label-correcting-algo.ipynb):

In [7]:
def label_correcting_algo(dt, ori_node, des_node, do_return=False):
    """
    Find the shortest path from Origin to Destination under a constant-link-costs network.
    
    Args:
        dt: Network representation. At least 3 columns:
              "start": start nodes of links
              "end": end nodes of links
              "cost": constant costs of links
        ori_node: Origin node.
        des_node: Destination node.
        do_return: Boolean. 
            If True, Return a dataframe as described below.
            If False, Return the list of nodes on the shortest path.
        
    Returns:
        A dataframe of two columns:
            "Front-Node": The node visited before current node on the shortest path.
            "Distance": Total distance from origin to current node.
    """
    # Convert all labels to string
    ori = str(ori_node)
    des = str(des_node)
    dt[["start", "end"]] = dt[["start", "end"]].astype(str) 
    
    # Initialization
    nodes = set(dt.loc[:,"start"].unique()) | set(dt.loc[:,"end"].unique())
    dist = {}.fromkeys(nodes, np.inf)
    dist[ori] = 0
    points = {}.fromkeys(nodes, ori)
    iter_set = {ori}
    
    # Main Algo
    while iter_set:
        i = iter_set.pop()  # Randomly pop out a node i
        A_i = dt[dt.start == i]
        for row in A_i.index: 
            j = A_i.loc[:, "end"][row]
            c_ij = A_i.loc[:, "cost"][row]
            if dist[j] > dist[i] + c_ij:
                dist[j] = dist[i] + c_ij
                points[j] = i
                iter_set = iter_set | set([j])  # Union
    
    # Print & Return the Answer
    x = pd.concat([pd.Series(points), pd.Series(dist)], axis=1)
    x.columns = ["Front-node", "Costs"]

    current_node = des
    front_node = ""
    sp = [des]
    while front_node != ori:
        front_node = str(x.loc[current_node, "Front-node"])
        # sp = "{} -> {}".format(front_node, sp)
        sp.insert(0, front_node)
        current_node = front_node
    
    # sp.append(x.loc[des, "Costs"])
    # sp = "From node {} to node {}, total cost: {}\n{}\n".format(ori, des, x.loc[des, "Costs"], sp)
    if do_return:
        print(sp)
        return x
    else:
        return sp

### Q1

*Find the user equilibrium (UE) flow pattern, and compute the total travel time experienced by all users (this is not to be confused with the sum of the integral of travel times). Plot the convergence curves (total network link flow vs iteration number, total network travel time vs iteration number) and discuss your results.*

-----


First, we use free flow travel time to find the initial shortest path. So the cost under this situation equals free flow travel time:

In [8]:
bprlinkarr = linkarr.copy()
bprlinkarr["cost"] = bprlinkarr.fftime
del(bprlinkarr["type"])

bprlinkarr.head()

,start,end,fftime,cap,cost
0,1,2,10,50,10
1,2,3,10,50,10
2,3,4,10,50,10
3,14,15,10,50,10
4,15,16,10,50,10


Define a function to implement the algorithm:

1. Initialization
2. Do AON assignment for every O-D pair based on current link costs
3. Load those flow on the network, and re-compute costs on each link
4. Do the line search and find the numerical solution of "k"
5. Update the flow on each link by $x_a + k\times direction$
6. Check the err. If err > thre, go to step 2; otherwise, next.
7. Return a dataframe with assignments in each iteration.

The object function in iteration $i$ is:

$$ \newcommand{\ud}{\,\mathrm{d}}
z_1(k) = \sum_a \int_0^{x_a^{(i-1)} + k\times d_a^{(i)}} t_a(x)\ud x $$

It should have a minimal between [0, 1]. Then:

$$\exists k^*, \quad\textrm{s.t. } \left.\frac{\ud z_1}{\ud k}\right|_{k=k^*} = 0$$

Therefore, we have (omit iteration label $i$):

$$
\sum_a t_a^0 \left[1 + \alpha\left(\frac{x_a + k^*d_a}{c_a'}\right)^\beta\right]\cdot d_a = 0
$$

Simplify the coeffecients and put them into the function defined below:

In [9]:
def ue_coef(df, d, xa_idx_str, alpha=0.15):
    p4 = alpha * sum(np.multiply(df.fftime, np.divide(d ** 5, (0.9 * df.cap) ** 4)))
    p3 = alpha * 4 * sum(np.multiply(df.fftime, np.divide(np.multiply(df[xa_idx_str], d ** 4), (0.9 * df.cap) ** 4)))
    p2 = alpha * 6 * sum(np.multiply(df.fftime, np.divide(np.multiply(df[xa_idx_str] ** 2, d ** 3), (0.9 * df.cap) ** 4)))
    p1 = alpha * 4 * sum(np.multiply(df.fftime, np.divide(np.multiply(df[xa_idx_str] ** 3, d ** 2), (0.9 * df.cap) ** 4)))
    p0 = sum(alpha * np.multiply(df.fftime, np.divide(np.multiply(d, df[xa_idx_str] ** 4), (0.9 * df.cap) ** 4)) + np.multiply(df.fftime, d))
    
    return [p0, p1, p2, p3, p4]

Then we define the main function:

In [10]:
def frank_wolfe(x, OD_arr, thre=1, alpha=0.15):
    ori_lst = OD_arr.index
    des_lst = OD_arr.columns
    err = thre + 1
    iters = 0
    x.cost = x.fftime  # Double-check for s.p. algo
    
    while (err > thre):
        new_col = "xa{}".format(iters)
        x[new_col] = 0
        x["cost{}".format(iters)] = 0
        
        # AON assignment
        for i in range(len(ori_lst)):
            ori = ori_lst[i]
            for j in range(len(des_lst)):
                des = des_lst[j]
                demand = OD_arr.at[ori, des]
                splst = label_correcting_algo(x, ori, des)
                while len(splst) > 1:
                    current_node = splst.pop()
                    front_node = splst[-1]
                    x.loc[(x.start == front_node) & (x.end == current_node), new_col] += demand
        
        # Line search
        if (iters > 1):
            direction = x[new_col] - x[old_col]
            
            # Compute the coefficients of the equation of k. 0 <= k <= 1.
            coef_lst = ue_coef(x, direction, old_col)
            k = solvex4eq(coef_lst)
            if (type(k) == type([])):
                print("Solving error. Iteration: {}".format(iters))
                return k
            x[new_col] = x[old_col] + k * (x.loc[:, new_col] - x[old_col])
            
            # Compute the error
            err = np.sqrt(sum((x[new_col] - x[old_col]) ** 2)) / sum(x[old_col])

        # Update the costs
        # xa-[iters] is computed based on cost-[iters] (so cost1 == fftime)
        x["cost{}".format(iters)] = x.cost
        x.cost = bpr(x[new_col], x.fftime, x.cap)
        
        if (iters > 50):
            print("Too many iterations.")
            return -1000
        
        # Move forward
        old_col = "xa{}".format(iters)
        iters += 1
    
    # End of while loop
    print("Total iteration: {}. Error: {:.4f}.".format(iters, err))
    return x

Here's the solution for Q1:

In [11]:
q1 = frank_wolfe(bprlinkarr, ODarr, thre=0.005)
q1

Total iteration: 10. Error: 0.0040.


,start,end,fftime,cap,cost,xa0,cost0,xa1,cost1,xa2,...,xa5,cost5,xa6,cost6,xa7,cost7,xa8,cost8,xa9,cost9
0,1,2,10,50,752.087015,550,10,0,33482.793781,240.576172,...,183.144976,670.970590,201.416076,421.548711,239.287565,612.029439,224.314622,1209.283123,212.228355,936.129128
1,2,3,10,50,1242.756907,700,10,0,87838.074989,407.128906,...,252.379060,2819.294896,274.672681,1494.070315,281.128179,2092.107128,258.890501,2294.855420,240.940086,1653.257672
2,3,4,10,50,574.118857,550,10,0,33482.793781,240.576172,...,159.972595,385.488853,179.397788,249.565109,222.881360,388.886858,209.206174,912.684944,198.167465,710.711416
3,14,15,10,50,436.318884,450,10,0,15010.000000,203.564453,...,202.601129,1176.679321,214.922752,626.323388,172.886777,790.496175,194.806866,336.804721,184.766547,536.815138
4,15,16,10,50,680.423389,600,10,0,47417.407407,370.117188,...,212.478762,1421.387161,231.779136,755.595556,172.702618,1065.695235,222.322821,335.414490,206.907860,903.670416
5,16,9,10,50,1012.126017,600,10,0,47417.407407,259.082031,...,236.790181,2186.901689,254.879733,1159.995033,215.403551,1553.769361,245.825731,797.503763,228.781173,1345.828058
6,8,1,15,200,82.448319,0,15,1200,15.000000,755.859375,...,445.549020,80.956980,423.358591,99.464550,442.893169,83.853497,407.859627,97.468562,421.181860,74.311017
7,1,9,15,200,1181.732036,100,15,1850,15.214335,1165.283203,...,912.404044,1201.998389,871.942514,1500.392763,853.605604,1253.921850,833.545005,1152.945672,858.953505,1049.686327
8,9,10,15,200,63.443006,0,15,1150,15.000000,724.365234,...,449.194225,95.632000,426.822247,102.262802,369.009155,86.134567,379.370736,54.741208,387.734679,59.396405
9,10,9,15,200,15.000000,0,15,0,15.000000,0.000000,...,0.000000,15.000000,0.000000,15.000000,0.000000,15.000000,0.000000,15.000000,0.000000,15.000000


Draw a animation graph to show the result:

In [12]:
links = np.arange(len(q1)) + 1
iters = (q1.shape[1] - 5) // 2
iterlst = list(map(lambda x: "xa{}".format(x), np.arange(iters)))
y = list(np.array(q1[iterlst].T))

# Draw animation graphs
fig, ax = plt.subplots()
fig.set_tight_layout(True)

ax.plot(links, q1.iloc[:, -2], 'r-', linewidth=2)
plt.hold(True)
line, = ax.plot(links, links - 5, 'k-')
ax.plot([28, 28], [0, 2000], 'k--')
ax.text(30, 1500, "Dummy links")
ax.axis(ymin=0)
plt.hold(False)
#line, = ax.plot(links, links - 5, 'r-', linewidth=2)

def update(i):
    global q1
    line.set_ydata(q1.loc[:, "xa{}".format(i)])
    ax.set_xlabel("Iteration {}".format(i))
    return line, ax

anim = FuncAnimation(fig, update, np.arange(iters), interval=1*1000)
anim.save(r'{}/pics/FrankWolfe-q1-linkflow.gif'.format(os.getcwd()), dpi=100, writer='imagemagick')
plt.close('all')

Here's the animation graph:

<img src="https://raw.githubusercontent.com/wklchris/Reports/master/pics/FrankWolfe-q1-linkflow.png" alt="Network Picture" style="width: 700px;"/>

Of course, we can draw figures of total flow/travel time vs iteration: